# Sistemas dinámicos discretos: iteración

Empezaremos con lo más básico: mapeos iterados como sistemas dinámicos discretos. 

Dadas una función $\mathbf{f}_\mathbf{\mu}: \mathbb{R}^n \to \mathbb{R}^n$, con parámetro(s) $\mathbf{\mu}$, y una condición inicial $\mathbf{x}_0$, calculamos los iterados

$$\mathbf{x}_1 = \mathbf{f}_\mathbf{\mu}(\mathbf{x}_0); \quad \mathbf{x}_2 = \mathbf{f}_\mathbf{\mu}(\mathbf{x}_1); \quad \ldots \quad \mathbf{x}_{n+1} = \mathbf{f}_\mathbf{\mu}(\mathbf{x}_n), \ldots.$$

Podemos pensar en $n$ como el tiempo discreto (número de pasos).


## Mapeo logístico

Veamos cómo implementar esto en Julia, con uno de los mapeos más conocidos, el **mapeo logístico** en 1 dimensión:

$$f(\mu, x) := \mu x (1 - x).$$

### Definir una función
Primero, necesitamos definir la función. Para funciones sencillas matemáticas, Julia provee una sintaxis sencilla:

In [ ]:
                                        logistico(μ, x) = μ * x * (1 - x)

Nota que Julia permite utilizar símbolos de Unicode como variables y operadores. E.g. para poner μ, tecleamos `\mu<TAB>`, donde `<TAB>` indica que apretemos la tecla "tabulador".

La función se ejecuta de la siguiente manera:

In [ ]:
logistico(0.5, 0.3)

## Iteración: bucle `for`

Ahora queremos implementar la iteración de una función con un número de pasos dado, para lo cual utilizamos un bucle `for`. Vamos guardando los datos en un vector 1-dimensional con la función `push!`:

In [ ]:
"""Iterar la función `f` desde la condición inicial `x0`, un número `n` de veces"""
function iterar(f, x0, n)
    iterados = [x0]
    x = x0
    
    for i in 1:n
        x = f(x)
        push!(iterados, x)
    end
    
    return iterados
end

Nota que `f` es un argumento que utilizamos como función adentro de la función `iterar`. La función `f` debe aceptar unicamente un argumento (ya que así lo utilizamos).

Probémoslo con una función sencilla:

In [ ]:
g(x) = 0.5*x

In [ ]:
iterar(g, 0.1, 10)

Sin embargo, no lo podemos utilizar con la función `logistico`, la cual acepta *dos* argumentos; nos arroja un error, que nos indica dónde se encuentra el problema - en este caso en la función `iterar`, en la línea 7. 

[Pista: Utiliza `Esc-L` dentro de una celda de código para agregar los números de línea.
Todas estas combinaciones de teclas se enlistan en `Help->Keyboard Shortcuts` en el menú del notebook.]

In [ ]:
iterar(logistico, 0.1, 10)

Nota que debemos tener cuidado con los números en la computadora: los números reales se representan con números de "punto flotante", los cuales se guardan en un espacio de un cierto tamaño finito en la memoria (64 bits, usualmente). Por lo tanto, *solo ciertos números reales se pueden representar con los números flotantes*.

Además, debido a la estructura interna de los números flotantes, pueden ocurrir cosas extrañas:

#### Ejercicio

Itera el **mapeo de doblamiento**, $f(x) := 2x \, \mathrm{mod}\,  1$, utilizando un `if` o `%` para el módulo. Empieza desde distintas condiciones iniciales, por ejemplo aleatorias (con `rand()`). ¿Qué observas?

## Funciones anónimas

Lo que queremos hacer es fijar el *parámetro* $\mu$, y tratar a la función 
$$x \mapsto f_\mu(x)$$ 
como una función de una sola variable.

Julia nos permite hacerlo con una sintaxis muy similar, utilizando una **función anónima**. Además, debemos almacenar los datos, al crear una nueva variable que 

In [ ]:
iterar(x->logistico(0.5, x), 0.7, 10)

Nota que la variable `iterados` que existe adentro de la función `iterar` *no existe afuera de la función*:

In [ ]:
iterados

Es una **variable local**. Para tener acceso a los datos desde fuera de la función, debemos asignar un nombre al objeto que se regresa:

In [ ]:
iterados = iterar(x->logistico(0.5, x), 0.7, 10)

## Graficar

Ahora que hayamos generado unos datos, los quisiéramos visualizar. Utilizaremos el **paquete** (librería / biblioteca) de Julia `Plots.jl`, ya que permite utilizar distintas librerías gráficas con una sola sintaxis.

Si no utilizas JuliaBox, se baja e instala el paquete con

In [ ]:
Pkg.add("Plots")

Esta instalación se hace una única vez en cada instalación de Julia.

Ahora para cargar la biblioteca, ponemos 

In [ ]:
using Plots

Esto se hace en cada trabajo que requiere el paquete. La primera vez que se utiliza, se tendrá que precompilar muchas funciones, por lo cual puede ser tardado.

In [ ]:
plot(iterados)

Dado que son datos discretos, deberíamos graficarlos más bien con puntos:

In [ ]:
scatter(iterados)
xlabel!("n", fontsize=50)
ylabel!("x_n")

## Interactividad

Julia provee herramientas que permiten lograr un grado de interactividad de forma fácil, a través del paquete `Interact.jl`:

In [ ]:
using Interact

Provee un "macro" (algo así como una "super-función") `@manipulate`, que toma un bucle `for` y lo convierte en una visualización interactiva:

In [ ]:
@manipulate for μ in 0.0:0.01:4.0
    iterados = iterar(x->logistico(μ, x), 0.7, 100)
    
    scatter(iterados)
    ylims!(0, 1)

end

Podemos agregar otro slider:

In [ ]:
@manipulate for μ in 0.0:0.01:4.0, N in 1:1000
    iterados = iterar(x->f(μ, x), 0.7, N)
    
    scatter(iterados)
    ylims!(0, 1)

end

#### Ejercicio

Utiliza la visualización para entender cuáles son los comportamientos asintóticos posibles del sistema, es decir cuando $n \to \infty$.

## Diagrama de bifurcación

#### Ejercicio

Dibuja el famoso **diagrama de bifurcación**, que muestra $\mu$ en el eje $x$, y para cada valor de $\mu$ algunos cuantos iterados.

Pistas: 
- Haz un bucle sobre los valores de $\mu$.
- Para agregar un plot encima de otro, utiliza `scatter!` con `!` al final.
- La función `scatter` en general acepta dos vectores, uno de coordenadas $x$ y otro de coordenadas $y$. Para generar las coordenadas $x$, puedes utilizar la función `ones(v)`, la cual crea un vector de unos de la misma longitud de `v`.

## Medida invariante

#### Ejercicio

Encuentra una aproximación numérica de la **medida invariante natural**, la cual muestra la frecuencia con la cual la dinámica visita cada parte del espacio.

Para hacerlo, utiliza la función `histograma` de `Plots.jl`. Utiliza `bins=100` para cambiar el número de cajas.

¡Hazlo interactivo!